# Makemore - Activations & Gradients, BatchNorm

## Imports

In [2]:
import torch 
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt 

%matplotlib inline
#%config InlineBackend.figure_format='retina'
%load_ext autoreload
%autoreload 2

In [3]:
SEED = 2147483647

## Dataset

In [4]:
# reading the dataset 
words = open('name.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [9]:
# Look up for characters
chars = sorted(set(''.join(words)))
stoi = {s : i + 1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i : s for s, i in stoi.items()} 

# Dictornary key, value and in enumerate index, value (column_names, values)

In [18]:
# Dataset 
def build_dataset(words):
    block_size = 3
    context = []
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
        
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y
            

In [19]:
random.seed(42)
random.shuffle(words)

n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182437, 3]) torch.Size([182437])
torch.Size([22781, 3]) torch.Size([22781])
torch.Size([22928, 3]) torch.Size([22928])


## Model